# Optimize a portfolio with the Quantum Portfolio Optimizer Qiskit function

This exercise demonstrates how to use Qiskit’s quantum portfolio optimizer function to solve financial portfolio optimization problems. We show how to formulate and solve dynamic portfolio optimization problems in a simple and accessible way, making it suitable for execution on a quantum computer or simulator with no quantum computing expertise required. The objective is to illustrate how quantum algorithms apply to real-world financial problems using intuitive tools and workflows provided by Qiskit.

In this exercise, we also show how to fine-tune the quantum portfolio optimizer settings. Although this fine-tuning is not necessary for basic usage, these advanced options provide insights into how experienced users can leverage quantum computing to improve efficiency and accuracy. For further details, consult the [documentation](https://docs.quantum.ibm.com/guides/global-data-quantum-optimizer) of the global data quantum portfolio optimizer.


## Table of Contents

- [Function Description](#function-description)
- [DPO Job Execution Example](#dpo-job-execution-example)
- [Exercise 1: DPO Job Execution](#exercise-1-dpo-job-execution)
- [Exercise 2: Resuming Job Execution](#exercise-2-resuming-job-execution)


## Setup

In [ ]:
# Install dependencies
%pip install "qiskit[visualization]"~=2.1.0 qiskit-serverless~=0.24.0 qiskit-ibm-catalog~=0.8.0 yfinance==0.2.60 pandas==2.1.4

In [ ]:
# Imports
import yfinance as yf
import pandas as pd
from qiskit_ibm_catalog import QiskitFunctionsCatalog
from grader import grade_ex1a, grade_ex1b, grade_ex2

<div class="alert alert-block alert-warning">

**Exclusive Access to Qiskit Functions**

As part of Qiskit Global Summer School (QGSS), participants with a Premium or Flex Plan have limited-time trial access to Qiskit Functions. Access is exclusive and subject to your organization’s administrator approval. Complete [this form](https://airtable.com/appj8IrSNZGz4l4BB/pag8WgWdUr5uSJGZA/form) to request access.

If you encounter the error `QiskitServerlessException: Credentials couldn't be verified`. in the cell below, it means your access to Qiskit Functions is not yet active. Please check back later after your request has been processed.

**Note: Running this lab will consume QPU time from your organization’s account. Estimated QPU usage is provided before each cell that executes on a QPU. Please monitor your usage and consult your organization admin if you’re unsure about your allocated QPU time for QGSS Functions labs.**

</div>

In [ ]:
# Load the Qiskit Functions Catalog
your_api_key = "deleteThisAndPasteYourAPIKeyHere"
your_crn = "deleteThisAndPasteYourCRNHere"

catalog = QiskitFunctionsCatalog(
    channel="ibm_quantum_platform",
    token=your_api_key,
    instance=your_crn,
)
# You should see a list of Qiskit Functions available to you
# If you encounter the error `QiskitServerlessException: Credentials couldn't be verified`,
# it means your access is not yet active
catalog.list()

In [ ]:
# Load Global Data Quantum Quantum Portfolio Optimizer function
dpo_solver = catalog.load("global-data-quantum/quantum-portfolio-optimizer")

## Function Description

The dynamic portfolio optimization problem involves determining the optimal investment strategy over multiple time periods in order to maximize the expected return of the portfolio and minimize risks, often under certain constraints such as transaction costs, or risk aversion. Unlike standard portfolio optimization, which considers a single time to rebalance the portfolio, the dynamic version accounts for the evolving nature of assets and rebalance the investment based on changes in asset performance over time.

To solve the dynamic portfolio optimization problem, we formulate it as a QUBO (Quadratic Unconstrained Binary Optimization) problem. In this approach, the variables are discretized based on the number of assets in the portfolio, the number of time steps considered, and the number of resolution bits used to define the investment strategy.

Following the formulation described in our [manuscript](https://arxiv.org/pdf/2412.19150), the QUBO problem is framed as a multi-objective optimization task, aiming to maximize the expected return, minimize risks, and reduce transaction costs (expenses associated with changing positions over time). Additionally, we introduce a penalty term to enforce the maximum investment per asset.

The final goal is to obtain a binary string as a solution, indicating how much to invest in each asset at each point in time. To illustrate this, consider a simplified case with 3 assets and 3 time steps. 

| Date       | META (%) | AAPL (%) | GOOGL (%) |
|------------|----------|----------|------------|
| 2024-07-01 | 16.67    | 50.00    | 33.33      |
| 2024-08-01 | 50.00    | 50.00    | 0.00       |
| 2024-09-01 | 42.86    | 42.86    | 14.29      |



## DPO Job Execution Example

In the cells below, we show how to solve a dynamic portfolio optimization problem using the quantum portfolio optimizer Qiskit Function. Specifically, we model and solve a three-period portfolio allocation problem involving three financial assets. The optimization can be performed using a binary encoding with a resolution of two bits, providing a simple yet insightful framework to understand how to leverage the capabilities of a quantum investment portfolio optimizer. This exercise is designed to introduce key concepts in quantum finance while leveraging Qiskit’s functions for implementing quantum algorithms in a practical financial context.

First, we have to load the historical data of the assets. For this example, we  build our portfolio using three major technology companies: Meta Platforms Inc. (ticker: META), Apple Inc. (ticker: AAPL), and Alphabet Inc. (ticker: GOOGL). These assets serve as the basis for constructing and optimizing our portfolio over three time periods.

To work with the data effectively, it must be structured as a JSON object that maps each asset's ticker symbol to a dictionary of closing prices by date. Each date should follow the YYYY-MM-DD format, and prices can be either normalized or raw. All assets must share the same set of dates to ensure consistency; if any asset is missing data for a given date, the missing value should be filled—typically using forward fill with the last known price.

To simplify the process, we use the provided function, which only requires the date range and the list of asset tickers. It automatically downloads the data, aligns the dates, fills missing values, and returns the data in the correct JSON format.

<a id="Example"></a>
<div class="alert alert-block alert-success">

<b>Example:</b> Follow the example to learn how to use the tool. It is not necessary to execute it, but doing so can help confirm that everything is correctly configured.

</div>


In [ ]:
def load_asset_data(symbols, start_date, end_date):
    """
    Downloads and prepares historical close price data for the given list of asset symbols.
    Also includes weekends by forward-filling the last known value.

    Parameters:
    - symbols (list of str): Ticker symbols (e.g., ['META', 'AAPL', 'GOOGL'])
    - start_date (str): Start date in 'YYYY-MM-DD' format
    - end_date (str): End date in 'YYYY-MM-DD' format

    Returns:
    - assets (dict): Dictionary representation of the DataFrame with prices per date and symbol
    """
    series_list = []
    symbol_names = [symbol.replace(".", "_") for symbol in symbols]

    # Create a full date index including weekends
    full_index = pd.date_range(start=start_date, end=end_date, freq='D')

    for symbol, name in zip(symbols, symbol_names):
        print(f"Downloading data for {symbol}...")
        data = yf.download(symbol, start=start_date, end=end_date)["Close"]
        data.name = name

        # Reindex to include weekends
        data = data.reindex(full_index)

        # Fill missing values (e.g., weekends or holidays) by forward/backward fill
        data.ffill(inplace=True)
        data.bfill(inplace=True)

        series_list.append(data)

    # Combine all series into a single DataFrame
    df = pd.concat(series_list, axis=1)

    # Convert index to string for consistency
    df.index = df.index.astype(str)

    # Convert DataFrame to dictionary
    assets = df.to_dict()
    return assets

<div class="alert alert-block alert-info">

<b>Tip:</b> 

You can reuse this function to efficiently solve the upcoming exercises.

</div>

Now it's time to define the date range over which we want to obtain historical data for our assets. To do this, we first need to specify the time window considered in each time step (`dt`). This is important because we need, at a minimum, the closing prices for ``(nt + 1) * dt`` days, where `nt` is the number of time steps in our portfolio optimization problem.

The time window (`dt`) we use is one month (30 days). Since our problem has 3 time steps (`nt = 3`), we need data covering 4 months in total. For example, we collect data from July 1, 2022, to November 1, 2022.

In [ ]:
# Define the list of asset symbols 
symbols = [
    "META", "AAPL", "GOOGL",    
]
# Define the start and end dates for the portfolio data
start_date = "2024-07-01"
end_date = "2024-11-01"

# get the asset data dictionary
assets = load_asset_data(symbols, start_date, end_date)

Next, we need to define the maximum amount to invest at each time step. Since we are using a 2-bit resolution, the maximum investment amount per time step cannot exceed `(2**(nq) - 1) * n_assets  = 9`. So for this case we fix the maximum amount to 7 (i.e., we allow for a maximum investment per asset of 7/9 ~ 77%).

In [ ]:
# define max investment parameter
max_investment = 7

Since we use the Differential Evolution algorithm as our classical optimizer, we need to define the number of generations and the population size (number of individuals) for the optimization process.

Note that the total amount of circuits is ``(num_generations + 1) * population_size``. In this case, to avoid taking large computation time,  we execute 60 circuits.

In [ ]:
# define the number of generations and the population size
num_generations = 5
population_size = 10

Finally, we just need to set all the required parameters and pass them appropriately into the quantum portfolio optimizer function.

In [ ]:
nt = 3 # Define the number of time steps
nq = 2 # Define the number of resolution bits
dt = 30 # Define the time window size

max_parallel_jobs = 3 # Define the amount of parallel jobs executed in the QPU. Maximum parallel jobs available for open plan is 3.
max_batchsize = 4 # Define the number of circuits per job. Note that estimator_shots*max_batchsize should be less than 10_000_000.

estimator_shots = 5_000 # Define the number of shots for the estimator. 
sampler_shots = 10_000 # Define the number of samples of the optimized circuit.

ansatz = 'real_amplitudes' # Define the ansatz to be used in the optimization
multiple_passmanager = False # Specify not using  multiple passmanager option 

apply_postprocess = True # Specify if apply SQD-Based postprocess. 

backend_name = None # Chooses the least busy backend available for the instance.

qubo_settings = {
    'nt': nt,
    'nq': nq,
    'dt': dt,
    'max_investment': max_investment,
}

optimizer_settings = {
    'de_optimizer_settings': {
        'num_generations': num_generations,
        'population_size': population_size,
        'max_parallel_jobs': max_parallel_jobs, 
        'max_batchsize': max_batchsize,
    },
    'optimizer': 'differential_evolution', 
    'primitive_settings':  {
        'estimator_shots': estimator_shots,
        'sampler_shots': sampler_shots,
    }                
}

ansatz_settings = {
    'ansatz': ansatz,
    'multiple_passmanager': multiple_passmanager,
}

<div class="alert alert-block alert-warning">

**⚠️ Warning: QPU Time Consumption**

Running the cell below will submit a job to a QPU and consume real QPU time. Please ensure you intend to proceed.

**Estimated QPU runtime:** 3 minutes 50 seconds (based on tests on `ibm_brussels`)

</div>

In [ ]:
dpo_job = dpo_solver.run(
    assets=assets, 
    qubo_settings=qubo_settings, 
    optimizer_settings=optimizer_settings, 
    ansatz_settings=ansatz_settings, 
    backend_name=backend_name, 
    apply_postprocess=apply_postprocess
)

In [ ]:
# Get the results of the job
dpo_result = dpo_job.result()

# Show the solution strategy
dpo_result['result']

Next we show how to access the metrics associated with the solution of the job. Specifically, we access the following metrics: Deviation from maximum investment, Sharpe ratio, and investment return.

In [ ]:
# Convert metadata to a DataFrame, excluding 'session_id'
df = pd.DataFrame(dpo_result['metadata']['all_samples_metrics'])

# Find the minimum objective cost
min_cost = df['objective_costs'].min()

# Extract the row with the lowest cost
best_row = df[df['objective_costs'] == min_cost].iloc[0]

# Display the results associated with the best investment
print("Best Investment Strategy:")
print(f"  - Deviation from maximum investment: {best_row['rest_breaches']}%")
print(f"  - Sharpe Ratio: {best_row['sharpe_ratios']:.2f}")
print(f"  - Return: {best_row['returns']}")

## Exercise 1: DPO Job Execution
In this exercise, we perform a portfolio optimization using three financial assets, four time steps of 30 days of time window, and a 2-bit resolution. This time, we use the Optimized Real Amplitudes Ansatz. The population size and number of generations are chosen to ensure that a total of 70 quantum circuits are executed during the optimization process. Additionally, we allow for a maximum investment per asset of 6/9 (approximately 66%).

The portfolio selected for this exercise are:
- [NVIDIA Corporation](https://finance.yahoo.com/quote/NVDA/)
- [Tesla, Inc.](https://finance.yahoo.com/quote/TSLA/)
- [Amazon.com, Inc.](https://finance.yahoo.com/quote/AMZN/)

<a id="Exercise1"></a>
<div class="alert alert-block alert-success">

<b>Exercise 1:</b> Follow the instructions in the cells below to perform portfolio optimization.

</div>


<a id="tips"></a>
<div class="alert alert-block alert-info">
    
<b>Tips:</b> 

Visit the links to the asset pages on Yahoo Finance to check the ticker names.

</div>


In [ ]:
### TODO: Write your code below here ###

# Fill the missing asset tickers of the portfolio
symbols = []

# Define the QUBO problem specification parameters (nt, nq, dt, )
#  - Set max_investment to approximately 66% of maximum investment per asset. (See example above)
#  - Set nt to four time steps.
#  - Set nq to two resolution bits.
#  - Set dt to 30 days.

qubo_settings = {
    'nt': ,  # Number of time steps
    'nq': ,  # Number of resolution bits
    'dt': ,  # Time window size in days
    'max_investment': ,  # Set max investment to approximately 66% of maximum investment per asset
}

# Define the end dates for the portfolio data so that it fills the required amount of days according to the time window size and the number of time steps.
start_date = "2024-07-01"
end_date = ""

# get the asset data dictionary

In [ ]:
# Knowing that we want to run at most 70 quantum circuits:
# set the population size and the number of generations for the Differential Evolution algorithm accordingly. 
# remember that the number of circuits is calculated as (num_generations + 1) * population_size.

### TODO: Write your code below here ###

num_generations = 
population_size = 

In [ ]:
max_parallel_jobs = 3 # Define the amount of parallel jobs executed in the QPU. Maximum parallel jobs available for open plan is 3.
max_batchsize = 4 # Define the number of circuits per job.

estimator_shots = 25_000 # Define the number of shots for the estimator. 
sampler_shots = 100_000 # Define the number of samples of the optimized circuit.

# Now complete the configuration by defining the remaining parameters. 
# Remember We use:
#  - Optimized Real Amplitudes ansatz. 
#  - Not enable the multiple passmanager option. 
#  - Apply Postprocessing based on SQD. 
#  - The backend with the least load available to ensure more efficient execution.

### TODO: Write your code below here ###

optimizer_settings_ex1 = {
    'de_optimizer_settings': {
        'num_generations': ,
        'population_size': ,
        'max_parallel_jobs': , 
        'max_batchsize': ,
    },
    'optimizer': 'differential_evolution', 
    'primitive_settings':  {
        'estimator_shots': ,
        'sampler_shots': ,
    }                
}

ansatz_settings = {
    'ansatz': ,
    'multiple_passmanager': ,
}

In [ ]:
grade_ex1a(qubo_settings, optimizer_settings_ex1, ansatz_settings, apply_postprocess, assets)

In [ ]:
your_api_key = "deleteThisAndPasteYourAPIKeyHere"
your_crn = "deleteThisAndPasteYourCRNHere"

catalog = QiskitFunctionsCatalog(
    channel="ibm_quantum_platform",
    token=your_api_key,
    instance=your_crn,
)
dpo_solver = catalog.load("global-data-quantum/quantum-portfolio-optimizer")

<div class="alert alert-block alert-warning">

**⚠️ Warning: QPU Time Consumption**

Running the cell below will submit a job to a QPU and consume real QPU time. Please ensure you intend to proceed.

**Estimated QPU runtime:** 4 minutes 20 seconds (based on tests on `ibm_brussels`)

</div>

In [ ]:
# Execute the job.

### TODO: Write your code below here ###

dpo_job = dpo_solver.run(
    assets= ,
    qubo_settings= , 
    optimizer_settings= , 
    ansatz_settings= , 
    backend_name= , 
    apply_postprocess= ,
)

In [ ]:
dpo_job.status() # Check the status is DONE before getting the result

Now, let's display the following key performance metrics: return, Sharpe ratio, deviation from the investment restriction, and transaction costs.

In [ ]:
### TODO: Write your code below here ###

# Get the results of the job
dpo_result = 

In [ ]:
grade_ex1b(dpo_result)

In [ ]:

# Convert metadata to a DataFrame, excluding 'session_id'
df = pd.DataFrame(dpo_result['metadata']['all_samples_metrics'])

# Find the minimum objective cost
min_cost = df['objective_costs'].min()

# Extract the row with the lowest cost
best_row = df[df['objective_costs'] == min_cost].iloc[0]

# Display the results associated with the best investment
print("Best Investment Strategy:")
print(f"  - Deviation from maximum investment: {best_row['rest_breaches']}%")
print(f"  - Sharpe Ratio: {best_row['sharpe_ratios']:.2f}")
print(f"  - Transaction Costs: {best_row['transaction_costs']}")

## Exercise 2: Resuming Job Execution
This function allows you to resume a previous execution (either because it was interrupted or because you want to perform additional runs to improve the result). In this exercise, we resume the previous optimization and add two more generations to continue refining the portfolio solution

To do this, we use the argument `previous_session_id`, which is a list of session IDs from which the execution is being resumed. Then, we need to provide exactly the same parameters as in the previous function call, but with two additional generations compared to the original example.

<a id="Exercise2"></a>
<div class="alert alert-block alert-success">

<b>Exercise 2:</b> Modify the number of generations to perform a warm restart and extend the execution.

</div>


In [ ]:
num_generations_ex1 = optimizer_settings_ex1['de_optimizer_settings']['num_generations']

In [ ]:
# First we take the session id from the previous job.
session_id = dpo_result['metadata']['session_id']

# Change the number of generations by adding 2 to the previous number of generations.

### TODO: Write your code below here ###

optimizer_settings = optimizer_settings_ex1
optimizer_settings['de_optimizer_settings']['num_generations'] = 

# Execute the job again introducing the new number of generations and the session id in the `previous_session_id` list.

### TODO: Write your code below here ###

previous_session_id =  # Load session id from the output of the previous exercise dpo_job.

In [ ]:
grade_ex2(qubo_settings, optimizer_settings, ansatz_settings, apply_postprocess, assets, num_generations_ex1, previous_session_id)

<div class="alert alert-block alert-warning">

**⚠️ Warning: QPU Time Consumption**

Running the cell below will submit a job to a QPU and consume real QPU time. Please ensure you intend to proceed.

**Estimated QPU runtime:** 1 minutes 30 seconds (based on tests on `ibm_brussels`)

</div>

In [ ]:
dpo_job = dpo_solver.run(
    assets=assets, 
    qubo_settings=qubo_settings, 
    optimizer_settings=optimizer_settings, 
    ansatz_settings=ansatz_settings, 
    backend_name=backend_name, 
    apply_postprocess=apply_postprocess,
    previous_session_id=previous_session_id
)

In [ ]:
dpo_job.status() # Check the status is DONE 

# Feedback Survey

We’d love to hear about your experience using the Qiskit Function! Your feedback is valuable and will help Qiskit Function providers enhance their tools and services. Please take a moment to share your thoughts by completing our short 2 min [feedback survey](https://airtable.com/app6VujlNUHZuOnAF/pagpw6TgP9UEt4TAT/form).

# References

1. [Quantum Portfolio Optimizater Tutorial](https://quantum.cloud.ibm.com/docs/en/tutorials/global-data-quantum-optimizer)
2. [Quantum Portfolio Optimizer Documentation](https://quantum.cloud.ibm.com/docs/en/guides/global-data-quantum-optimizer)
3. [Scaling the Variational Quantum Eigensolver for Dynamic Portfolio Optimization](https://arxiv.org/abs/2412.19150)
4. [Qiskit Serverless Documentation](https://qiskit.github.io/qiskit-serverless/index.html)

# Additional Information
**Created by**: Manuel Martín-Cordero, Álvaro Nodar  
**Advised by**: Junye Huang

**Version**: 1.0.0

## Qiskit packages versions

In [ ]:
import qiskit
import qiskit_ibm_catalog

print(f'Qiskit: {qiskit.__version__}')
print(f'Qiskit IBM Catalog: {qiskit_ibm_catalog.__version__}')